In [ ]:
import numpy as np

### 概率求解：给定观测序列和模型参数计算观测序列出现的概率
1. 前向算法：根据初值计算后一时刻观测值的概率
    - 初值
    $$\alpha(i)=pi_ib_i(o_1), i=1,2,3,...N$$
    - 递推 对t=1, 2, 3, ..., T - 1
    $$\alpha_{t+1} = \left[\sum_{j=1}^N \alpha_t(j)a_{ji} \right]b_i(o_{t+1}), i=1,2,...,N$$
    - 终止
    $$P(O|\lambda) = \sum_{i=1}^N \alpha_T(i)$$

In [125]:
# 例 10.2（p200页）
# 状态集合Q = {1, 2, 3},观测集合V={红，白} 0 1
Q = [1, 2, 3]
A = np.array([[0.5, 0.2, 0.3], [0.3, 0.5, 0.2], [0.2, 0.3, 0.5]])
B = np.array([[0.5, 0.5], [0.4, 0.6], [0.7, 0.3]])
Pi = np.array([0.2, 0.4, 0.4])
V = ['红', '白']    # 观测值集合
# 设T = 3，观测值O = {红 白 红}
T = 3   # 时间
N = 3   # 隐藏状态数量
O = ['红', '白', '红']   # 观测值

In [ ]:
# 1. 计算初值
alpha1 = Pi * B[:, 0]   # 初始观测值为0时的概率

# 2. 递推计算
alpha2 = np.array([(alpha1 * A[:, i]).sum() * B[i, 1] for i in range(N)])
alpha3 = np.array([(alpha2 * A[:, i]).sum() * B[i, 0] for i in range(N)])
# alpha4 = np.array([(alpha3 * A[:, i]).sum() * B[i, 0] for i in range(N)])

# 3. 终止
result = alpha3.sum()
print(alpha1)
print(alpha2)
print(alpha3)
print(result)

In [ ]:
# 封装函数
def forward(obs, V, Pi, A, B):
    """ 前向算法计算概率值，知道当前时刻观测值的概率，计算下一个时刻观测值的概率"""
    T = len(obs)
    N = len(Pi)
    # 计算初值
    alpha1 = Pi * B[:, 0]
    # 递推计算
    def cal_alpha(alpha, o):
        return np.array([(alpha * A[:, i]).sum() * B[i, o] for i in range(N)])
    
    for j in obs[1:]:
        o = V.index(j)
        alpha = alpha1
        alpha1 =  cal_alpha(alpha, o)
    # 返回终值
    return alpha1.sum()

forward(O, V, Pi, A, B)

2. 后向算法：根据最后时刻观测值的概率计算前一个时刻观测值的概率
    - 初值
    $$\beta_T(i) = 1, i=1, 2, ..., N$$
    - 对t=T-1, T-2, ...,1
    $$\beta_t(i)=\sum_{j=1}^N a_{ij}b_j(o_{t+1}) \beta_{t+1}(j), i=1, 2, ...,N$$
    - 终值
    $$P(O|\lambda)=\sum_{i=1}^N \pi_i b_i(o_1) \beta_1(i)$$

In [ ]:
# 计算T时刻的不同状态下的概率
beta3 = [1, 1, 1]     # 每列代表不同的状态
# 求T-1，即2时刻的概率,状态1,
beta2 = np.array([(A[i, :] * B[:, 0] * beta3).sum() for i in range(N)])
beta1 = np.array([(A[i, :] * B[:, 1] * beta2).sum() for i in range(N)])
# 求终值
p = (Pi * B[:, 0] * beta1).sum()
print(p)

In [ ]:
def backward(obs, V, Pi, A, B):
    """ 后向算法 """
    # 得到相关参数
    T = len(obs)
    N = len(Pi)
    
    # 最终时刻的概率
    last_beta = np.ones((3, ))
    def  cal_result(beta, o):
        return np.array([(A[i, :] * B[:, o] * beta).sum() for i in range(N)])
    
    for j in obs[::-1][:-1]:
        o = V.index(j)    # t+1时刻的状态值
        beta = last_beta
        last_beta = cal_result(beta, o)
    return (Pi * B[:, 0] * last_beta).sum()

print(backward(O, V, Pi, A, B))

---

### 预测算法：又称解码算法，即给定观测序列，求解最可能的状态序列
1. 近似算法：在每个时刻选择在该时刻最可能出现的状态，从而得到一个状态序列作为结果
2. 维特比算法：用动态规划求解最有路径
    - 初始化：
    $$\delta_1(i) = \pi_i * b_i(o_1), \;\;i=1, 2, ..., N$$
    $$\Psi_1(i) = 0,\;\; i=1, 2, ..., N$$
    - 递推。对t=2, 3, ..., T
    $$\delta = \mathop{max}\limits_{1\leq j \leq N} [\delta_{t-1}(j) a_{ji}]b_i(o_t),\;i=1,2,...,N$$
    $$\Psi(i)=arg \mathop{max}\limits_{1\leq j \leq N}[\delta_{t-1}(j)a_{ji}],\; i=1,2,...,N$$
    - 终止
    $$P^* = \mathop{max}\limits_{1\leq j \leq N} \delta_T(i)$$
    $$i_T^* = arg \mathop{max}\limits_{1\leq j \leq N}[\delta_T(i)]$$
    - 最优路径回溯，对t=T-1,T-2,..., 1
    $$i_t^* = \Psi_{t+1}(i_{t+1}^*)$$
    最优路径$I^* = (i_1^*,\; i_2^*, \;..., \;i_T^*)$

In [123]:
# 例 10.3（P210)
# 1.初始值
idx_o = V.index(O[0])
delta1 = Pi * B[:, idx_o]    # 初始状态分布概率乘以观测值发射概率
psi1 = np.zeros((3, ))    # 状态集合

# 2. t=2时，
idx_o2 = V.index(O[1])
delta2 = np.array([(delta1 * A[:, i]).max() * B[i, idx_o2] for i in range(N)])
psi2 = np.array([np.argmax((delta1 * A[:, i]) * B[i, idx_o2]) for i in range(N)])

# t = 3 时
idx_o3 = V.index(O[2])
delta3 = np.array([(delta2 * A[:, i]).max() * B[i, idx_o3] for i in range(N)])
psi3 = np.array([np.argmax((delta2 * A[:, i]) * B[i, idx_o3]) for i in range(N)])

# 3. 最优路径的概率
P = delta3.max()
i3 = np.argmax(delta3)    # 最优路径终点

# 回溯最优路径
i2 = psi3[i3]
i1 = psi2[i2]
I = (i1, i2, i3)

In [206]:
def viterbit(obs, Q, V, Pi, A, B):
    # 构建一个行为状态数，列为观测值数的矩阵
    N = len(Q)
    F = np.zeros((N, len(obs)))
    P = np.zeros((N, len(obs)))
    
    # 计算初始值
    F[:, 0] = Pi * B[:, V.index(obs[0])]
    
    # 计算其他值
    for j, o in enumerate(obs[1:], 1):
        idx = V.index(o)
        F[:, j] = np.array([(F[:, j - 1] * A[:, i] * B[i, idx]).max() for i in range(N)])
        P[:, j] = np.array([(F[:, j - 1] * A[:, i] * B[i, idx]).argmax() for i in range(N)])
    
    i = F[:, -1].argmax()    # 终点的最优路径
    I = [i]
    for l in range(len(obs) - 1, 0, -1):
        i = int(P[:, l][i])
        I.append(i)
    return [Q[i] for i in I[::-1]]

In [207]:
O1 = ["白", "白", "红", "红", '红', '白']
viterbit(O1, Q, V, Pi, A, B)

[2, 2, 3, 3, 3, 2]

In [150]:
from hmmlearn import hmm

In [156]:
model = hmm.MultinomialHMM(n_components=len(Q))
model.startprob_ = Pi
model.transmat_ = A
model.emissionprob_ = B

In [204]:
O = np.array([[1, 1, 0, 0, 0, 1]]).T
model.decode(O)

(-8.411644693428197, array([1, 1, 2, 2, 2, 1]))

---

### 学习算法：已知观测序列，估计模型的参数A，B，Pi。
- 包含观测序列和对应的状态序列：有监督学习。
- 仅包含观测序列：无监督学习

#### 监督学习方法
设训练数据包含S个长度相同的观测序列和对应的状态序列$\{(O_1, I_1),(O_2, I_2),...,(O_s, I_s)\}$,分词算法所使用的方法

#### 无监督学习（Baum-Wwlch算法)：E步求期望，M步求极大化
设训练数据包含S个长度为T的的观测序列$\{O_1,O_2,...,O_s\}$，求HMM模型的参数
1. 确定完全数据的对数似然函数是$\log P(O, I, \lambda)$
2. 算法实现步骤
    - 输入：观测数据$O = (o_1, o_2, ..., o_T)$
    - 输出：隐马尔可夫模型参数$\lambda$.
    - (1) 初始化。对n=0,选取$a_{ij}^{(0)}，b_j(k)^{(0)}，\pi_i^{(0)},得到模型\lambda^(0)=(A^{(0)}, b^{(0)}, \pi ^{(0)})$
    - (2) 递推。对n=1,2,...
    $$a_{ij}^{n+1} = \frac{\sum_{t=1}^{T-1} \xi_t(i, j)}{\sum_{t=1}^{T-1} \gamma_t(i)} $$
    $$b_j(k)^{n+1} = \frac{\sum_{t=1,o_t=\nu_k} \gamma_t{j}}{\sum_{t=1}^T \gamma_t(j)} \\$$
    $$\pi_i^{n+1} = \gamma_1(i)$$
    其中：
    $$\gamma_t(i)= \frac{\alpha_t(i)\beta_t(i)}{\sum_{j=1}^N \alpha_t(j)\beta_t(j)}\\ $$
    $$\xi_t(i, j) = \frac{\alpha_t(i)\alpha_{ij}b_j(o_{t+1})\beta_{t+1}(j) }{\sum_{i=1}^N \sum_{j=1}^N \alpha_t(i)\alpha_{ij}b_j(o_{t+1})\beta_{t+1}(j)}$$